In [1]:
import cnvpytor
import subprocess

print(cnvpytor.__version__)

1.3.1


In [2]:
filename = "test_with_del_400_5_formated.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip())
line_count

1135036

In [3]:
read_count = line_count * 50 * 30 / 202
read_count = round(read_count)
read_count

8428485

In [4]:
command = f"wgsim -d 300 -N {read_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

[wgsim] seed = 1700452087
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 56751720


b'chr22\t365\tA\tM\t+\nchr22\t1902\tT\tW\t+\nchr22\t3024\tT\tY\t+\nchr22\t3318\tA\tM\t+\nchr22\t4999\tC\tA\t-\nchr22\t6284\tG\tS\t+\nchr22\t6594\tT\t-\t-\nchr22\t6760\tC\tG\t-\nchr22\t7868\tC\tY\t+\nchr22\t8333\tT\tG\t-\nchr22\t8383\tC\tY\t+\nchr22\t9092\tC\tY\t+\nchr22\t11219\tC\tY\t+\nchr22\t11276\tG\tC\t-\nchr22\t12165\tC\tA\t-\nchr22\t14166\tA\tW\t+\nchr22\t14464\tG\tA\t-\nchr22\t14715\tG\tK\t+\nchr22\t16740\tG\tR\t+\nchr22\t18172\tC\tT\t-\nchr22\t19127\tT\tW\t+\nchr22\t19395\tG\tS\t+\nchr22\t20668\tC\tM\t+\nchr22\t21810\tA\tR\t+\nchr22\t22010\tT\tY\t+\nchr22\t22389\tA\tW\t+\nchr22\t24731\tT\tY\t+\nchr22\t24995\tC\tS\t+\nchr22\t25131\tG\tK\t+\nchr22\t28456\tG\tC\t-\nchr22\t32935\tT\t-\t-\nchr22\t33773\tA\tG\t-\nchr22\t35107\tG\tC\t-\nchr22\t38022\tC\t-\t-\nchr22\t38837\tA\tC\t-\nchr22\t41065\tG\tR\t+\nchr22\t44007\tC\tS\t+\nchr22\t44139\tC\tY\t+\nchr22\t44541\tA\tT\t-\nchr22\t46391\tA\tT\t-\nchr22\t47899\tT\tK\t+\nchr22\t48266\tT\tA\t-\nchr22\t49047\tC\t-\t-\nchr22\t49059\t-\tC\t+\

In [5]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:01
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [6]:
# sam to bam file conversion
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam

[bam_sort_core] merging from 10 files and 1 in-memory blocks...


In [7]:
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)

In [8]:
app.rd(["biasOut_sorted.bam"])

In [9]:
app.calculate_histograms([100, 1000, 10000, 100000, 1000000, 10000000])

In [10]:
app.partition([100, 1000, 10000, 100000, 1000000, 10000000])

In [11]:
calls = app.call([100, 1000, 10000, 100000, 1000000, 10000000], print_calls=True)

duplication	chr22:643201-644800	1600	1.4291	2.473043e-05	6.940632e-65	1.000000e+00	1.000000e+00	0.0000	1.0000	0
duplication	chr22:1306101-1325100	19000	1.3441	0.000000e+00	6.391408e-241	0.000000e+00	1.215627e-214	0.0000	1.0000	0
duplication	chr22:2926201-2963000	36800	3.0133	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:4546401-4713300	166900	1.6024	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6655001-6661800	6800	2.3362	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6848301-6854600	6300	1.6079	0.000000e+00	1.531045e-268	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:8039401-8047700	8300	1.8814	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:9424001-9461600	37600	1.4298	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:13230601-13395900	165300	1.3538	0.000000e+00	0.000000e+00	0.0000

In [12]:
for bin_size in calls:
    print("Bin size: ", bin_size)

Bin size:  100
Bin size:  1000
Bin size:  10000
Bin size:  100000
Bin size:  1000000
Bin size:  10000000


In [13]:
print(f"{filename=}, Histogram Size: 100")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_5_formated.fa', Histogram Size: 100
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,643201,644800,1600,1.4290609648593058,0.0
duplication,chr22,1306101,1325100,19000,1.3440670234745071,0.0
duplication,chr22,2926201,2963000,36800,3.0132546605939217,0.0
duplication,chr22,4546401,4713300,166900,1.602434607171792,0.0
duplication,chr22,6655001,6661800,6800,2.3362231959782864,0.0
duplication,chr22,6848301,6854600,6300,1.607917528238872,0.0
duplication,chr22,8039401,8047700,8300,1.881412000950326,0.0
duplication,chr22,9424001,9461600,37600,1.4297911799093308,0.0
duplication,chr22,13230601,13395900,165300,1.3537577592704642,0.0
duplication,chr22,13480901,13512800,31900,4.076203464650919,0.0
duplication,chr22,15242601,15246400,3800,2.90285166268524,0.0
duplication,chr22,18662501,18664600,2100,1.5013591477739334,0.0
duplication,chr22,18664801,18665900,1100,1.536933813910517,0.0
duplication,chr22,18680801,18682700,1900,1.4719206946213848,0.0
duplicat

In [14]:
print(f"{filename=}, Histogram Size: 1000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_5_formated.fa', Histogram Size: 1000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,1306001,1325000,19000,1.3450524335588785,0.0
duplication,chr22,2926001,2963000,37000,3.0065909463957716,0.0
duplication,chr22,4546001,4713000,167000,1.603548412692582,0.0
duplication,chr22,6655001,6662000,7000,2.304815518314925,0.0
duplication,chr22,8040001,8048000,8000,1.8758709383896488,0.0
duplication,chr22,9424001,9462000,38000,1.4274356086929143,0.0
duplication,chr22,13231001,13396000,165000,1.3551499097181512,0.0
duplication,chr22,13481001,13514000,33000,3.9775744327389453,0.0
duplication,chr22,18705001,18881000,176000,2.8307061227598482,0.0001409699844933017
duplication,chr22,19259001,19268000,9000,3.299749801238494,0.0
duplication,chr22,20609001,20637000,28000,1.4968209446549812,0.00019653186764233822
duplication,chr22,21248001,21381000,133000,1.5518808805623976,0.00012213659384936043
duplication,chr22,21497001,21624000,127000,1.3935199965360836,0.

In [15]:
print(f"{filename=}, Histogram Size: 10,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_5_formated.fa', Histogram Size: 10,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,4550001,4710000,160000,1.5983849914776793,0.0
duplication,chr22,13230001,13390000,160000,1.3498181051879399,0.0
duplication,chr22,18710001,18890000,180000,2.736341464724917,0.00016275996305571797
duplication,chr22,21250001,21380000,130000,1.5333235657014697,0.00012208985270537547
duplication,chr22,21520001,21660000,140000,1.3707928386939623,0.0004311914161496891
duplication,chr22,24200001,24240000,40000,2.629760480410148,4.703693444619236e-05
duplication,chr22,25150001,25310000,160000,1.3099944546690574,0.00012919239095966458
duplication,chr22,28470001,28600000,130000,3.124366717902081,4.274712318755645e-05
duplication,chr22,30800001,30980000,180000,3.043092271768072,2.4985520890643873e-05
duplication,chr22,31150001,31360000,210000,1.4725873754834213,4.9661151322848916e-05
duplication,chr22,31870001,32250000,380000,1.3037400062225946,7.964465211050032e-0

In [16]:
print(f"{filename=}, Histogram Size: 100,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_5_formated.fa', Histogram Size: 100,000
type,chromosome_name,start,end,size,cnv_level,q0


In [17]:
print(f"{filename=}, Histogram Size: 1,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_5_formated.fa', Histogram Size: 1,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [18]:
print(f"{filename=}, Histogram Size: 10,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_5_formated.fa', Histogram Size: 10,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [19]:
del app